In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp pod.db

In [ ]:
# export
# hide
class DB():
    def __init__(self):
        self.nodes = dict()

    def add(self, node):
        uid = node.id
        if uid in self.nodes:
            raise ValueError(f"Trying to add node, but node with with id: {uid} is already in database")
        self.nodes[uid] = node
        
    def update(self, node):
        uid = node.id
        if uid not in self.nodes:
            raise ValueError(f"Trying to update node, but node with with id: {uid} is not in database")
        self.nodes[uid] = node

    def get(self, id):
        res = self.nodes.get(id, None)
        return res

    def contains(self, node):
        id = node.get_property("id")
        return id in self.nodes

    def create(self, node):
        existing = self.get(node.properties.get("id", None))

        if existing is not None:
            if not existing._expanded:
                existing.edges = node.edges
                existing._expanded = node.edges is not None
            return existing
        else:
            self.add(node)
            return node

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted basic.ipynb.
Converted cvu.utils.ipynb.
Converted data.photo.ipynb.
Converted importers.Importer.ipynb.
Converted importers.util.ipynb.
Converted index.ipynb.
Converted indexers.indexer.ipynb.
Converted itembase.ipynb.
Converted plugin.pluginbase.ipynb.
Converted plugin.schema.ipynb.
Converted plugin.stateful.ipynb.
Converted plugin.states.ipynb.
Converted plugins.authenticators.password.ipynb.
Converted pod.client.ipynb.
Converted pod.db.ipynb.
Converted pod.utils.ipynb.
Converted test_utils.ipynb.
